Data Processing Notebook
===

Prepare data for the analysis. The raw data is downloaded from the FAIR Universe HiggsML challenge repository. Use the HiggsML package to download and process the dataset, followed by selections and saving to local cache.

NB: This notebook is only to be run once to get the FAIR Universe dataset into `.root` ntuples. The rest of the workflow is independent of this.


In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mplhep as hep
import yaml
import uproot

from utils import plot_kinematic_features

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import HiggsML
from HiggsML.systematics import systematics
hep.style.use(hep.style.ATLAS)

from HiggsML.datasets import download_dataset

In [2]:
data = download_dataset("https://zenodo.org/records/15131565/files/FAIR_Universe_HiggsML_data.zip")

2025-11-10 17:44:45,732 - HiggsML.datasets     - INFO     - Handling as URL: https://zenodo.org/records/15131565/files/FAIR_Universe_HiggsML_data.zip
2025-11-10 17:44:45,736 - HiggsML.datasets     - INFO     - Current working directory: /home/jsandesara_umass_edu/NSBI-workflow-tutorial/FAIR_universe_Higgs_tautau
2025-11-10 17:44:45,965 - HiggsML.datasets     - INFO     - Total rows: 220099101
2025-11-10 17:44:45,966 - HiggsML.datasets     - INFO     - Test size: 66029730


In [3]:
data.load_train_set(train_size=0.35)
df_training_full = data.get_train_set()
del data

2025-11-10 17:44:54,525 - HiggsML.datasets     - INFO     - Selected train size: 53924279
2025-11-10 17:51:04,104 - HiggsML.datasets     - INFO     - Data loaded successfully


In [4]:
list_of_processes_to_model = ["htautau", "ztautau", "ttbar"]

In [5]:
process_to_exclude = set(df_training_full["detailed_labels"].unique()) - set(list_of_processes_to_model)
process_to_exclude = list(process_to_exclude)
print(process_to_exclude)

['diboson']


In [6]:
mask_process_exclusion = ~np.isin(df_training_full["detailed_labels"], process_to_exclude)

df_training_full = df_training_full[mask_process_exclusion].copy()
df_training_full["detailed_labels"].value_counts()

detailed_labels
ztautau    34425566
htautau    17854072
ttbar       1515091
Name: count, dtype: int64

In [7]:
# Trim the dataset, so all processes have equal entries

# get the number of ttbar events (lowest)
n_ttbar = df_training_full.loc[
    df_training_full.detailed_labels=='ttbar'
].shape[0]

# Trim the other processes to match ttbar number, preserving event weight sums
df_list = []
for _, df_process in df_training_full.groupby('detailed_labels'):

    weight_sum_orig = df_process.weights.sum()

    df_sampled = df_process.sample(n = n_ttbar, random_state=42)

    df_sampled['weights'] *= weight_sum_orig / df_sampled['weights'].sum()

    df_list.append(df_sampled)
    
    del df_sampled

df_training = pd.concat(df_list).reset_index(drop=True)
del df_training_full, df_list

In [8]:
syst_settings = {
    'TES_up': {'tes': 1.02, 'seed': 42},
    'TES_dn': {'tes': 0.98, 'seed': 42},
    'JES_up': {'jes': 1.02, 'seed': 42},
    'JES_dn': {'jes': 0.98, 'seed': 42}
}


dataset_dict = {}

dataset_dict['nominal'] = systematics(
        data_set = df_training,
        dopostprocess=False
        )

for sample_name, syst_args in syst_settings.items():
    dataset_dict[sample_name] = systematics(
        data_set = df_training, 
        dopostprocess=False, 
        **syst_args
    )


In [9]:
saved_datasets = "./saved_datasets/"

In [10]:
# Some common analysis selections to remove low-stats regions
selections = "DER_mass_transverse_met_lep <= 250.0 and \
            DER_mass_vis <= 500.0 \
            and DER_sum_pt <= 1000 and \
            DER_pt_tot <= 250 and \
            DER_deltar_had_lep <= 4.5 and \
            DER_pt_h <= 400 and \
            DER_pt_ratio_lep_had <= 9.0"

In [11]:
for sample in dataset_dict.keys():

    # Write to ROOT TTree
    with uproot.recreate(f"{saved_datasets}dataset_{sample}.root") as ntuple:

        for process in list_of_processes_to_model:

            df = dataset_dict[sample]
            
            df_process = df[df["detailed_labels"] == process].copy()

            df_process = df_process.query(selections).copy()

            columns_to_keep = df_process.columns.tolist()

            columns_to_keep = list(set(columns_to_keep) - set(["detailed_labels"]))

            arrays = {col: df_process[col].to_numpy() for col in columns_to_keep}

            ntuple[f"tree_{process}"] = arrays